In [31]:
import os

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
import tensorflow as tf

%matplotlib inline

In [34]:
# constants
IMG_HEIGHT, IMG_WIDTH = 100, 100
BATCH_SIZE = 32
STEPS_PER_EPOCH = 50
VALIDATION_STEPS = 20
EPOCHS = 5
DATA_PATH  = 'data'
# couch_path = os.path.join(DATA_PATH, 'couch')
# DATA_PATH = os.path.join(os.getcwd(),'data')
# DATA_PATH = os.path.join(os.getcwd(),'data')
couch_path = os.path.join(DATA_PATH, 'couch') # 2016 couch pictures, to be augmented 
chair_path = os.path.join(DATA_PATH, 'chair') # 8141 chair pictures

couch_pictures = os.listdir(couch_path)
chair_pictures = os.listdir(chair_path)

In [55]:
# couchaug_path = os.path.join(DATA_PATH, 'couch_aug')
# couchaug_pictures=os.listdir(couchaug_path)
# print(len(couchaug_pictures))
print(len(couch_pictures))


2016


In [57]:
print(len(os.listdir(Final_Data)))

1000


In [20]:
# generator_factory = ImageDataGenerator(
#         rotation_range=10, 
#         width_shift_range=0.2,
#         height_shift_range=0.2,            
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

In [21]:
# couch_dir = os.path.join(DATA_PATH,'couch_aug')
# if not os.path.exists(couch_dir):
#     os.mkdir(couch_dir)

In [22]:
# for i in couch_pictures:
#     img = load_img(os.path.join(couch_path, i), target_size=(IMG_HEIGHT, IMG_WIDTH))
    
#     img = np.expand_dims(img, axis=0) #必须把第一维补上才能，第一维表示batch size，才能在下一行indicate batch size为1
#     img_generator = generator_factory.flow(img, batch_size=1, save_to_dir=couch_dir, save_format='jpg')
#     for j in range(3):
#         sampled_transformed_img = next(img_generator) # next一次生成一张照片，存在save_to_dir=couch_dir里面
#         sampled_transformed_img = np.squeeze(sampled_transformed_img, axis=0) #再把batch所表示的第一维去掉
#         #sampled_transformed_img = array_to_img(sampled_transformed_img)

In [23]:
# couchaug_path = os.path.join(DATA_PATH, 'couch_aug')
# couchaug_pictures=os.listdir(couchaug_path)
# print(len(couchaug_pictures))

In [24]:
print(len(couch_pictures))
print(len(couch_pictures))

0
0


In [36]:
# train-validation split

def train_val_split():
    couch_train, couch_valtest = train_test_split(couch_pictures, train_size=0.9, test_size=0.1)
    chair_train, chair_valtest = train_test_split(chair_pictures, train_size=0.9, test_size=0.1)
    couch_val, couch_test = train_test_split(couch_valtest, train_size=0.5, test_size=0.5)
    chair_val, chair_test = train_test_split(chair_valtest, train_size=0.5, test_size=0.5)

    

    if not os.path.exists('train'):
        os.mkdir('train')
        os.mkdir('train/couch')
        os.mkdir('train/chair')
        for filename in couch_train:
            os.rename(os.path.join(couch_path, filename), os.path.join('train/couch', filename))
        for filename in chair_train:
            os.rename(os.path.join(chair_path, filename), os.path.join('train/chair', filename))

    if not os.path.exists('val'):
        os.mkdir('val')
        os.mkdir('val/couch')
        os.mkdir('val/chair')
        for filename in couch_val:
            os.rename(os.path.join(couch_path, filename), os.path.join('val/couch', filename))
        for filename in chair_val:
            os.rename(os.path.join(chair_path, filename), os.path.join('val/chair', filename))
     
    
    if not os.path.exists('test'):
        os.mkdir('test')
        os.mkdir('test/couch')
        os.mkdir('test/chair')
        for filename in couch_test:
            os.rename(os.path.join(couch_path, filename), os.path.join('test/couch', filename))
        for filename in chair_test:
            os.rename(os.path.join(chair_path, filename), os.path.join('test/chair', filename))
            
if not os.path.exists('train') and not os.path.exists('test') and not os.path.exists('val'):
    train_val_split()
    
TRAIN_DATA, TEST_DATA = os.path.join(os.getcwd(),'train'), os.path.join(os.getcwd(),'test')
VALIDATION_DATA=os.path.join(os.getcwd(),'val')

In [26]:
TRAIN_DATA

'/app/notebooks/tutorials/train'

In [53]:
Final_Data=os.path.join(os.getcwd(),'final_test')

In [54]:
Final_Data

'/app/notebooks/tutorials/final_test'

In [37]:
# sample augmentations of the couch pictures
def setup_generator(data_dir):
    generator_factory = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.2,
            height_shift_range=0.2,            
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            preprocessing_function=preprocess_input)
    generator = generator_factory.flow_from_directory(
            data_dir, target_size=(IMG_HEIGHT, IMG_WIDTH))
    return generator

train_generator = setup_generator(TRAIN_DATA)
val_generator = setup_generator(VALIDATION_DATA)
text_generator = setup_generator(TEST_DATA)


Found 9140 images belonging to 2 classes.
Found 508 images belonging to 2 classes.
Found 509 images belonging to 2 classes.


In [58]:
Final_Data_generator=setup_generator(Final_Data)

Found 0 images belonging to 0 classes.


In [ ]:
# Large images width and heigth

from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization
def setup_convnet():
    """Set up a convolutional net for digit classification"""
    img = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    conv1 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(img)
    conv2 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv2)
    img_features = Flatten()(pool1)
#     =GlobalAveragePooling2D(pool1)
#     dense1 = Dense(64, activation='relu')(img_features)
    dense1 = Dense(64, activation='relu',kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform')(img_features)
#     dense2=BatchNormalization()(dense1)
    dropout2 = Dropout(0.3)(dense1)
    classifier = Dense(2, activation='softmax')(dropout2)
    model = Model(inputs=img, outputs=classifier)
    return model

model = setup_convnet()
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

train_history = model.fit_generator(train_generator, 
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    epochs=40,
                                    validation_data=val_generator,
                                    validation_steps=VALIDATION_STEPS,
                                    callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss')],
                                    class_weight={0:1,1:4})


res2 = model.evaluate_generator(text_generator)
res3 = model.evaluate_generator(val_generator)
# , steps=72, max_q_size=10, workers=1, pickle_safe=False)
print('Result of testing data: ')
print(model.metrics_names, '\n', res2)
print('Result of training data: ')
print( model.metrics_names, '\n', res3)

predictions = model.predict_generator(text_generator)
predictions_val = model.predict_generator(val_generator)
predicted_classes = np.argmax(predictions, axis=1)
predicted_val_classes = np.argmax(predictions_val, axis=1)

true_classes = text_generator.classes
true_val_classes = val_generator.classes
class_labels = list(text_generator.class_indices.keys())
class_val_labels = list(val_generator.class_indices.keys())


from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
report_val = classification_report(true_val_classes, predicted_val_classes, target_names=class_val_labels)
print("Result of testing:")
print (report)
print("Result of validation: ")
print(report_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 64)                294976    
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
__________

In [52]:
# Large images width and heigth

from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization
def setup_convnet():
    """Set up a convolutional net for digit classification"""
    img = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    conv1 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(img)
    conv2 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv2)
    img_features = Flatten()(pool1)
#     =GlobalAveragePooling2D(pool1)
#     dense1 = Dense(64, activation='relu')(img_features)
    dense1 = Dense(64, activation='relu',kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform')(img_features)
#     dense2=BatchNormalization()(dense1)
    dropout2 = Dropout(0.3)(dense1)
#     classifier = Dense(2, activation='softmax')(img_features)
    classifier = Dense(2, activation='softmax')(dropout2)
#     classifier = Dense(2, activation='sigmoid')(dropout2)
    model = Model(inputs=img, outputs=classifier)
    return model

model = setup_convnet()
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

train_history = model.fit_generator(train_generator, 
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    epochs=5,
                                    validation_data=val_generator,
                                    validation_steps=VALIDATION_STEPS,
                                    callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss')],
                                    class_weight={0:1,1:4})


res2 = model.evaluate_generator(text_generator)
res3 = model.evaluate_generator(val_generator)
# , steps=72, max_q_size=10, workers=1, pickle_safe=False)
print('Result of testing data: ')
print(model.metrics_names, '\n', res2)
print('Result of training data: ')
print( model.metrics_names, '\n', res3)

predictions = model.predict_generator(text_generator)
predictions_val = model.predict_generator(val_generator)
predicted_classes = np.argmax(predictions, axis=1)
predicted_val_classes = np.argmax(predictions_val, axis=1)

true_classes = text_generator.classes
true_val_classes = val_generator.classes
class_labels = list(text_generator.class_indices.keys())
class_val_labels = list(val_generator.class_indices.keys())


from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
report_val = classification_report(true_val_classes, predicted_val_classes, target_names=class_val_labels)
print("Result of testing:")
print (report)
print("Result of validation: ")
print(report_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_36 (Dense)             (None, 64)                294976    
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
__________

In [59]:
TRAINED_WEIGHTS_FILENAME = 'model1.hdf5'
model.save(TRAINED_WEIGHTS_FILENAME)
model = load_model(TRAINED_WEIGHTS_FILENAME)

In [60]:
datagen = ImageDataGenerator()
data_generator = datagen.flow_from_directory(
            Final_Data,
            target_size=(100, 100),
            batch_size=batch_size,
            shuffle = False, # Important !!!
            seed = 59,
            classes = None,
            class_mode = None)
image_list = data_generator.filenames
image_list = [x.split('/')[1].split('.')[0] for x in image_list]


In [70]:
from keras.callbacks import ModelCheckpoint
def setup_convnet():
    """Set up a convolutional net for digit classification"""
    img = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    conv1 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(img)
    conv2 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv2)
    #dropout1 = Dropout(0.25)(pool1)
#     conv3 = Conv2D(64, kernel_size=(30, 30), strides=1, activation='relu')(pool1)
#     conv4 = Conv2D(64, kernel_size=(30, 30), strides=1, activation='relu')(conv3)
# #     pool2 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv4)
    img_features = Flatten()(pool1)
#     =GlobalAveragePooling2D(pool1)
#     dense1 = Dense(64, activation='relu')(img_features)
    dense1 = Dense(64, activation='relu')(img_features)
    dropout2 = Dropout(0.3)(dense1)
#     classifier = Dense(2, activation='softmax')(img_features)
#     classifier = Dense(2, activation='softmax')(dropout2)
    classifier = Dense(2, activation='softmax')(dropout2)
    model = Model(inputs=img, outputs=classifier)
    return model

model = setup_convnet()
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

train_history = model.fit_generator(train_generator, 
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    epochs=EPOCHS,
                                    validation_data=val_generator,
                                    validation_steps=VALIDATION_STEPS,
                                    callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss')],
                                    class_weight={0:1,1:4})


res2 = model.evaluate_generator(text_generator)
res3 = model.evaluate_generator(val_generator)
# , steps=72, max_q_size=10, workers=1, pickle_safe=False)
print('Result of testing data: ')
print(model.metrics_names, '\n', res2)
print('Result of training data: ')
print( model.metrics_names, '\n', res3)

predictions = model.predict_generator(text_generator)
predictions_val = model.predict_generator(val_generator)
predicted_classes = np.argmax(predictions, axis=1)
predicted_val_classes = np.argmax(predictions_val, axis=1)

true_classes = text_generator.classes
true_val_classes = val_generator.classes
class_labels = list(text_generator.class_indices.keys())
class_val_labels = list(val_generator.class_indices.keys())


from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
report_val = classification_report(true_val_classes, predicted_val_classes, target_names=class_val_labels)
print("Result of testing:")
print (report)
print("Result of validation: ")
print(report_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                294976    
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
__________

In [58]:
def setup_convnet():
    """Set up a convolutional net for digit classification"""
    img = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    conv1 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(img)
    conv2 = Conv2D(32, kernel_size=(3, 3), strides=1, activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv2)
    #dropout1 = Dropout(0.25)(pool1)
#     conv3 = Conv2D(64, kernel_size=(30, 30), strides=1, activation='relu')(pool1)
#     conv4 = Conv2D(64, kernel_size=(30, 30), strides=1, activation='relu')(conv3)
# #     pool2 = MaxPooling2D(pool_size=(8, 8), strides=(8, 8))(conv4)
    img_features = Flatten()(pool1)
#     =GlobalAveragePooling2D(pool1)
#     dense1 = Dense(64, activation='relu')(img_features)
#     dense1 = Dense(64, activation='relu')(dense1)
#     dropout2 = Dropout(0.1)(dense1)
#     classifier = Dense(2, activation='softmax')(img_features)
#     classifier = Dense(2, activation='softmax')(dropout2)
    classifier = Dense(2, activation='softmax')(img_features)
    model = Model(inputs=img, outputs=classifier)
    return model

model = setup_convnet()
model.compile(loss=categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

train_history = model.fit_generator(train_generator, 
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    epochs=EPOCHS,
                                    validation_data=val_generator,
                                    validation_steps=VALIDATION_STEPS,class_weight={0:1,1:5})


res2 = model.evaluate_generator(text_generator)
res3 = model.evaluate_generator(val_generator)
# , steps=72, max_q_size=10, workers=1, pickle_safe=False)
print('Result of testing data: ')
print(model.metrics_names, '\n', res2)
print('Result of training data: ')
print( model.metrics_names, '\n', res3)

predictions = model.predict_generator(text_generator)
predictions_val = model.predict_generator(val_generator)
predicted_classes = np.argmax(predictions, axis=1)
predicted_val_classes = np.argmax(predictions_val, axis=1)

true_classes = text_generator.classes
true_val_classes = val_generator.classes
class_labels = list(text_generator.class_indices.keys())
class_val_labels = list(val_generator.class_indices.keys())


from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
report_val = classification_report(true_val_classes, predicted_val_classes, target_names=class_val_labels)
print("Result of testing:")
print (report)
print("Result of validation: ")
print(report_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 9218      
Total params: 19,362
Trainable params: 19,362
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [64]:
TRAINED_WEIGHTS_FILENAME = 'model_weights.hdf5'
model.save(TRAINED_WEIGHTS_FILENAME)
model = load_model(TRAINED_WEIGHTS_FILENAME)